# Fonction Pseudo-booléenne vers hamiltonian de coût pour QAOA 

Définison tout d'abord ce qu'est une fonction pseudo Booléenne. une fonction booléen normal prend la forme suivante : AB+BC+C(A!B) ceci voudrais dire "A et B OU B et C OU C et (A et NON B)" où chaque valeur A B C peuvent être soit "vrai" ou "faux". le résultat de la fonction est aussi "vrai" ou "faux".

une fonction pseudo-booléen est la même affaire, mais avec un coefficient imaginaire devant. par example : c<sub>0</sub>A-c<sub>1</sub>B*C où c<sub>k</sub> est un nombre imaginaire.

dans ce jupyter notebook, on va montrer comment transformer une fonction pseudo-booléen vers un hamiltonian de coût pour QAOA. cet hamiltonian peut après être pris pour trouver les valeus (A B C...) afin de minimiser les valeur de sorte à avoir le plus bas coût.

Tout d'abord, on définit une fonction qui prend en entrée une expression booléenne du genre :

"10\*A\*B+-10\*C+100\*!A"


où :

- `!` signifie **non** (négation),
- `+` signifie **ou** (disjonction),
- `*` signifie **et** (conjonction).

Cette fonction retourne une version de l'expression où les variables booléennes sont remplacées par des **projecteurs** quantiques.

---

On note :

- $ P_x = |x\rangle \langle x| $ est le projecteur sur l'état $ |x\rangle $,
- $ P_\tau |y\rangle = \delta_{\tau y} \, |y\rangle = y^\tau |y\rangle $  ($ y^\tau = 1 $ si $ y = \tau $, et 0 sinon).

Ainsi, on peut "remplacer" une variable booléenne $ A \in \{0,1\} $ par un projecteur $ P_A $, ce qui nous permet d'exprimer des fonctions booléennes comme des opérateurs agissant sur des états quantiques.

---

On utilise aussi le fait que, pour une base computationnelle $ |\vec{a}\rangle $, et une fonction booléenne $ f $, on a :

$$
\langle \vec{a} | f_H | \vec{a} \rangle = f(\vec{a})
$$

où $ f_H $ est l’opérateur (ou observable) représentant la fonction $ f $.

Autrement dit, appliquer $ f_H $ à un état $ |\vec{a}\rangle $ revient à multiplier cet état par la valeur $ f(\vec{a}) $.


In [ ]:
import re
import pennylane as qml
from pennylane import numpy as np
from pennylane import qaoa, AdamOptimizer, SPSAOptimizer, QNSPSAOptimizer, MomentumOptimizer, NesterovMomentumOptimizer, AdagradOptimizer, RMSPropOptimizer, RiemannianGradientOptimizer
from pennylane import *
import matplotlib.pyplot as plt

In [ ]:
def bool_expr_to_hamiltonian(expr, var_to_wire):
    expr = expr.replace(' ', '')
    terms = re.findall(r'([+-]?\d+(?:\.\d+)?)\*?([A-Za-z!*]*)', expr)

    def parse_var(v):
        negated = v.startswith('!')
        var_name = v[1:] if negated else v
        wire = var_to_wire[var_name]
        return f'p({wire},{0 if negated else 1})'

    hamiltonian_terms = []

    for coeff, vars_str in terms:
        if vars_str == '':
            term_expr = f'{coeff}*qml.Identity(0)'
        else:
            vars_list = vars_str.split('*')
            term_expr = ' @ '.join(parse_var(v) for v in vars_list)
            term_expr = f'{coeff}*{term_expr}'

        hamiltonian_terms.append(term_expr)

    return ' + '.join(hamiltonian_terms)


la fonction bool_expr_to_hamiltonian transforme les valeur A B C ... en projecteur $P_1^{qubit\_nb}$ et les valeur !A !B !C ... en projecteur $P_0^{qubit\_nb}$

In [ ]:
var_to_wire = {'A':0, 'B':1, 'C':2}

boolean_expression = "10*A*B+-10*C-100*!A"

hamiltonian_expr = bool_expr_to_hamiltonian(boolean_expression, var_to_wire)
hamiltonian_expr

nous savons que $P_x^{n} = \frac{1}{2} \left( I^{n} + (-1)^x Z^{n} \right)$

où :

- $P_x^{n}$  est le projecteur sur l’état associé à $ x \in \{0,1\} $ sur le $ n $ ième qubits,
- $ I^{n}$ est l'opérateur identité sur le $ n $ ième qubits,
- $ Z^{n}$ est l'opérateur de Pauli-Z agissant sur le $ n $ ième qubits

voici la fonction qui définis le projecteur $P_x^{n}$ en python :


In [ ]:
def p(wire,value):
    return (1/2)* (qml.Identity(wire)+((-1)**value)*qml.PauliZ(wire))

Notre chaine de charactère généré précedemment inclus l'appel à la fonction p(wire,value) qu'on vient de définir.
la porte identité et la porte pauliZ est implémenter avec pennylane. Le résultat: on peu généré notre hamiltonian de coût en utilisant la fonction eval de python. 

(la fonction eval est dangereuse niveau cybersecurité ! assurez-vous d'être sur du contenue de la chaine de charactère qui sera executer!)

In [ ]:
H_cost = eval(hamiltonian_expr)
H_cost

Testons maintenant notre hamiltonien de coût avec QAOA !


## Utilisation de QAOA avec pennylane pour trouver les valeurs minimum à notre fonction pseudo booléenne

In [ ]:
wires = range(len(var_to_wire)) # le nombre de qubits dans le circuit doit être aussi gros que le nombre de valeur booléen différentes!
dev = qml.device("lightning.qubit", wires=wires)

# Mixer Hamiltonian, on utilise x_mixer, c'est au goût
mixer_h = qaoa.x_mixer(wires)

In [ ]:
def qaoa_layer(gamma, alpha):
    qaoa.cost_layer(gamma, H_cost)
    qaoa.mixer_layer(alpha, mixer_h)

def circuit(params):
    for w in wires:
        qml.Hadamard(wires=w)
    for i in range(depth):
        qaoa_layer(params[i, 0], params[i, 1])

@qml.qnode(dev)
def cost_function(params):
    circuit(params)
    return qml.expval(H_cost)

maintenant, c'est le temps d'appliquer l'hamiltonian de coût avec QAOA et voir la performance avec différents optimizer et simulateurs.

In [ ]:
steps = 900
depth = 20
optimizer = AdamOptimizer()
params = np.random.uniform(0, np.pi, (depth, 2))
# Optimization loop
for i in range(steps):
    params = optimizer.step(cost_function, params)
    if i% 10 == 1:
        print(f'step {i} out of {steps}')

print("Optimal Parameters:")
print(params)
print("Optimal Energy:", cost_function(params))

investigation des méta paramètres utile pour SPSA

In [ ]:
steps = 20
depth = 20
first_params = np.random.uniform(0, np.pi, (depth, 2))
lowest_energy = 0
bestmeta = {'a':0.0,'b':0.0}
# Optimization loop
for a in np.arange(0.630, 0.632, 0.001, dtype=float):
    for c in np.arange(0.35, 0.355, 0.001, dtype=float):
        optimizer = SPSAOptimizer(maxiter=steps,c=c, a=a)
        params = first_params
        for i in range(steps):
            params = optimizer.step(cost_function, params)
            if i% 10 == 1:
                print(f'step {i} out of {steps}')
        result = cost_function(params)
        if result < lowest_energy:
            bestmeta['a'] = a
            bestmeta['c'] = c
            lowest_energy = result
        print(f"Optimal Energy: {result} a:{a} c:{c}")
#print("Optimal Parameters:")
#print(params)
#print("Optimal Energy:", cost_function(params))
print(f"best meta params : {bestmeta} with lowest energy {lowest_energy}")

In [ ]:
steps = 700
depth = 20
first_params = np.random.uniform(0, np.pi, (depth, 2))
lowest_energy = 0
# Optimization loop
a=0.631 # valeur trouver avec la cellule précédente
c=0.351 # valeur trouver avec la cellule précédente
optimizer = SPSAOptimizer(maxiter=steps, c=c, a=a)
params = first_params
for i in range(steps):
    params = optimizer.step(cost_function, params)
    if i% 10 == 1:
        print(f'step {i} out of {steps}')
print("Optimal Parameters:")
print(params)
print("Optimal Energy:", cost_function(params))


In [ ]:
steps = 70
depth = 20
optimizer = QNSPSAOptimizer()
params = np.random.uniform(0, np.pi, (depth, 2))
# Optimization loop
for i in range(steps):
    params = optimizer.step(cost_function, params)
    if i% 10 == 1:
        print(f'step {i} out of {steps}')

print("Optimal Parameters:")
print(params)
print("Optimal Energy:", cost_function(params))

In [ ]:
from scipy.optimize import minimize
def cost_function_wrapper(params_flat):
    params_reshaped = params_flat.reshape(depth, 2)
    return cost_function(params_reshaped)
depth = 20
steps = 70
params_flat = np.random.uniform(0, np.pi, depth*2)
params = minimize(cost_function_wrapper, params_flat, method='BFGS',
                  options={'maxiter':steps}
                  ).x.reshape(depth, 2)

print("Optimal Parameters:")
print(params)
print("Optimal Energy:", cost_function(params))

In [ ]:
from scipy.optimize import minimize
def cost_function_wrapper(params_flat):
    params_reshaped = params_flat.reshape(depth, 2)
    return cost_function(params_reshaped)
depth = 20
steps = 70
params_flat = np.random.uniform(0, np.pi, depth*2)
params = minimize(cost_function_wrapper, params_flat, method='COBYLA',
                  options={'maxiter':steps}
                  ).x.reshape(depth, 2)

print("Optimal Parameters:")
print(params)
print("Optimal Energy:", cost_function(params))

In [ ]:
from skopt import gp_minimize
from skopt.space import Real

def cost_function_wrapper(params_flat):
    params_reshaped = np.array(params_flat).reshape(depth, 2)
    return float(cost_function(params_reshaped))
result = gp_minimize(cost_function_wrapper, [Real(0, np.pi)] * (2 * depth), n_calls=25, random_state=42)
print("Optimal Parameters:", result.x)
print("Optimal Energy:", result.fun)
params = np.array(result.x).reshape(depth, 2)

In [ ]:
steps = 70
depth = 20
optimizer = MomentumOptimizer()
params = np.random.uniform(0, np.pi, (depth, 2))
# Optimization loop
for i in range(steps):
    params = optimizer.step(cost_function, params)
    if i% 10 == 1:
        print(f'step {i} out of {steps}')

print("Optimal Parameters:")
print(params)
print("Optimal Energy:", cost_function(params))

In [ ]:
import jax
import jax.numpy as jnp
from jaxopt import BFGS
steps = 70
depth = 20
def cost_function_wrapper(params_flat):
    params_reshaped = params_flat.reshape(depth, 2)
    return cost_function(params_reshaped)
# same thing but gpu
params_flat = np.random.uniform(0, np.pi, depth*2)
solver = BFGS(fun=cost_function_wrapper)
solution = solver.run(params_flat)

params = solution.params.reshape(depth, 2)
print("Optimal Parameters:")
print(params)
print("Optimal Energy:", cost_function(params))

In [ ]:
steps = 70
depth = 20
optimizer = QNGOptimizer()
params = np.random.uniform(0, np.pi, (depth, 2))
# Optimization loop
for i in range(steps):
    params = optimizer.step(cost_function, params)
    if i% 10 == 1:
        print(f'step {i} out of {steps}')

print("Optimal Parameters:")
print(params)
print("Optimal Energy:", cost_function(params))

In [ ]:
# Probability distribution of final state
@qml.qnode(dev)
def probability_circuit(params):
    circuit(params)
    return qml.probs(wires=wires)

probs = probability_circuit(params)

# Visualization
plt.style.use("seaborn-v0_8")
plt.bar(range(2 ** len(wires)), probs)
plt.xlabel('Basis State')
plt.ylabel('Probability')
plt.xticks(range(2 ** len(wires)), [f"{i:03b}" for i in range(2 ** len(wires))])
plt.title('Final State Probabilities after QAOA')
plt.show()

optimizer ranking with depth 20 and 70 steps and no noise: 

| Optimizer Name                                      | Avg Execution Time | Optimal Energy Found | Comment                                                                 |
|-----------------------------------------------------|---------------------|-----------------------|-------------------------------------------------------------------------|
| jax_BFGS                                            | 2m10s               | -109.97 Very reliable results              | supposedly only good in noiseless situations    |
| Adam                                                | 4.4s                | -108.55             |                                                                         |
| scipy_BFGS                                          | 1m3s                | -109.99 Very reliable results             |  supposedly only good in noiseless situations    |
| COBYLA                                              | 1.1s                | -96                   |                                                                         |
| gp_minimize (Bayesian optimization using GPs)       | 8.5s                | -98                   | Supposedly good in noisy situations                                     |
| QNSPSAOptimizer                                     | 38s                 | -95                   |                                                                         |
| SPSAOptimizer              | 14.5s               | -73  (very variable results)                  | Tuned meta parameter – supposedly good in noisy situations              |
| MomentumOptimizer                                   | 5s                  | -40 to -80 (variable) |                                                                         |
| QNGOptimizer                                        | 1m24s               | -41                   |                                                                         |


# Exemple d'une fonction booléenne intégrée dans une matrice et vérifié à la main
![Pseudo boolean mapping example](img/pseudo_boolean.jpg)

In [ ]:
import numpy as np
from functools import reduce

def bitstring_to_column_array(bitstring):
    """
    Converts a bitstring to a column NumPy array.

    Args:
        bitstring: The bitstring to convert.

    Returns:
        A column NumPy array representing the bitstring.
    """
    bit_list = [int(bit) for bit in bitstring]
    bit_array = np.array(bit_list)
    column_array = bit_array.reshape(-1, 1)
    return column_array

s1 = bitstring_to_column_array("10000000")
s2 = bitstring_to_column_array("00100000")
s3 = bitstring_to_column_array("00001000")
s4 = bitstring_to_column_array("00000100")
s5 = bitstring_to_column_array("00000010")
s6 = bitstring_to_column_array("00000001")
p_s_1 = s1*s1.T 
p_s_2 = s2*s2.T
p_s_3 = s3*s3.T
p_s_4 = s4*s4.T
p_s_5 = s5*s5.T
p_s_6 = s6*s6.T
print( p_s_1+p_s_2+2*p_s_3+p_s_4+3*p_s_5+2*p_s_6 )

ket0 = np.array([1, 0])
ket1 = np.array([0, 1])

p11 = np.outer(ket1, ket1)
p00 = np.outer(ket0, ket0)
iden2 = np.eye(2)

# helper
kron = lambda *mats: reduce(np.kron, mats)

term1 = kron(p11, iden2, iden2)          # |1⟩⟨1| ⊗ I ⊗ I₄
term2 = kron(iden2, iden2, p00)          # I ⊗ I₄ ⊗ |0⟩⟨0|
term3 = kron(np.kron(p11, p11), iden2)   # |11⟩⟨11| ⊗ I₄

result = term1 + term2 + term3           # each 8 × 8
print(result)